# Data Grand Prix: Building and Visualizing Formula 1 Stats with REST API

Searching for data sources to use for practicing or sharpening your skills could be tiresome. Data sources come in various forms but for this project I have focused on Rest API as a data source.I've often come accross or read articles that mention Rest APIs as a common source for retrieving data for exploration, but up until now, thanks to [Rapid API](https://rapidapi.com/) I  had never tried explored data with the source being an API. Here's my small little journey for a very basic process on how i ende up with some Formula 1 stats to explore. I hope you will find some useful tips along the way!

Before we jump in make sure you have access to to the following, feel free to use any equivalent tool of your choice that will satisfy the same purpose as intended in this project:
* Postman - went with Postamn to manage the API calls made to Rapid API but there are many other platforms that can do a similar job, feel free to use any of your choice.
* S3 Bucket - if you have gone with AWS or any equivalent service depending on which cloud service provider you have gone with.
* IAM Role with permissions to access S3 Bucket - necessary for executing jobs later, if you picked a different cloud service provider then the equivalence of this is also fine
* An Account with Rapid API - this is a free sign up and it even allows you a reasonable amount of API calls per day 
* IDE or Data Warehouse Account -  Myql Workbench/PostgreSql/MSSQL/Snowflake or any other of your choice

## Raw Data Architeture
Here is a quick visual to better understand what sort of Formula 1 stats we will be collecting from the data source:
![alt text](<05 archi-raw-f1.jpg>)
## Collect and Upload to S3 Bucket
### Phase 1 - Extracting Nested JSON Files From API 
Sign up with [Rapid API](https://rapidapi.com/) and go over to the marketplace and search for Formula 1. Pick the one made by API SPORTS.
Head over to Postman and create GET and PUT requests to call the data from Rapid AI and push to your S3 Bucket.
For the GET configuration make sure the headers provided by Rapid API are set up otherwise the request will not be returned.

![alt text](<01 GET_postman_config.png>)

Also ensure in the scripts tab on the *Post Response* section you put in this code ```javascript pm.environment.set("responseData", JSON.stringify(pm.response.json()));```.
This script saves the JSON response from the GET request to an environment variable called responseData. You will need the variable when setting up your PUT request.

![alt text](<02 GET_postman_config2.png>)

For the PUT request which is responsible for uploading your resukts from the GET request into the S3 Bucket, you would need to first create a [presigned URL](https://docs.aws.amazon.com/AmazonS3/latest/userguide/PresignedUrlUploadObject.html). You can choose a preferred method to do so as long as you use the generated URL in your PUT request it should work. I went wuth AWS SDK using Python as, here is an example of the script:

![alt text](<03 presigned URL eample.png>)

Another important piece to not miss out is adding the environment variable to the 'body' tab of your PUT request. If you miss this, you will upload a blank file into S3.

![alt text](<04 PUT_postman_config.png>)

If you already have a bucket set up with an IAM role, you should be good to go, if not, head to your AWS console and set it up. Important to not that you cannot generate a presigned URL if you do not have an existing bucket. 

Once all done run your requests in Postman starting with the GET request first then the PUT, this will upload the data into your bucket. Do this for all the APIs related to the data you require.

### Phase 2 - Transforming Nested JSON files to Parquet Format
Once I had results uploaded into the S3 Bucket, the next stage was to convert the data so that it was usable for what I wanted to do with it. As it stood the results were stored in json format. You could chose to download one file or preview results of your files from Postman to get an understanding of how your json files are structured. In my case some of the files had objects that were the "value" element of another object. This is otherwise known as nested json files.

To extract data that was nested with in another json object, i had to use Spark. Spark helped me to flatten the json nested json files by using a pyspak function called explode. Once that was done i was able to save the output in parquet format in a different bucket location to ensure that i have a seperate path to the raw data and the one that i had just processed.
I have pasted an example script i used to convert on of the files below: 

In [3]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, explode

# # Initialize Spark session builder
# spark_builder = SparkSession.builder.appName("MySparkSession")

# # Create the Spark session
# spark = spark_builder.getOrCreate()

# # Load JSON data from S3
# df = spark.read.format("json")\
#             .option("multiLine", True)\
#             .option("header", True)\
#             .option("inferschema", True)\
#             .load(s3_input_path)

# df.printSchema()

# print("done")
# # Explode the 'response' array
# exploded_df = df.withColumn("response", explode("response"))

# # Select the nested fields explicitly
# flattened_df = exploded_df.select(
#     col("response.capacity").alias("capacity"),
#     col("response.competition.id").alias("competition_id"),
#     col("response.competition.location.city").alias("competition_location_city"),
#     col("response.competition.location.country").alias("competition_location_country"),
#     ...,
#     col("response.length").alias("length"),
#     col("response.name").alias("name")
# )

# flattened_df.show(truncate=False)


# # Path to save the flattened data in S3
# s3_output_path = "s3a://your-s3-bucket-location/"

# # Write the flattened DataFrame to S3
# flattened_df.write.format("parquet").mode("overwrite").save(s3_output_path)

# print("complete")

### Phase 3 - Creating Tables from Parquet Files 
From the parquest files stored in a different S3 Bucket location, I now had the data in rows and colums format which meant i could now organise the data into tables. I used AWS GLue for this by firstly creating an AWS Crawler job. When you configuring teh crawler ensure you use the IAM role created dedicated for the job and that it also has permissions to access teh S3 Buckets. The Crawler ran through all the file paths where my parquest files were stored, processed the data to create schemas as intended. The output was as per the example below:

![alt text](<06 Data Catalog Tables.png>)

AWS has a service (AWS Athena) that allows you to have a look at the data and even query it once tables have been created in the Catalog. To access this straight was i just clicked on the Table Data link as ahown below and followed the prompt:

![alt text](<07 Data Catalog Tables View Athena.png>)

AWS Athena allwos you to even query your tables so you can have a quick look on what your processed files (from teh Crwler Job) look like. This is how one of my tables looked like:

![alt text](<08 Athena to query Data.png>)

All the tables looked good as expected. At this point you can choose to model how you want your tables to look like for example if there are seperate files for lets say different seasons, you could join the tables using some DDL queries and store your results in a clean fresh database which would be the database you then use to fetch your data from. Notice in the screenshot below on the left hand side how my tables have changed from the previous screenshot. I used some DDL commands in the `formula-1-database` which used tables that were stored in `formula-one` database. 

![alt text](<09 CTAS.png>)

### Phase 4 - Creating a RDS Instance and Mysql Connector
With our tables ready for further exploration, the options from this point depend on which tool or cloud service you would want to use tha data from. This will determine the next steps. In my case i wanted to connect this data to Mysql Workbench and from there i would use any other visualization tool to access the data. So the first step I took was to create an RDS instance then connect Mysql Workbench to this RDS Instance using a Mysql Connector. You can find more connectors on AWS marketplace depending on the type of connection desired. 

* Create an RDS Instance 
* Create a Mysql connector 

Meanwhile in Mysql workbench i created the exact same tables as the tables in the `formula-1-databse` database. The reason for this is explained in the next point.

As it stood i had tables from created from the first crawler i created plus the DDL queries i wrote only. In order to show the tables i created in Mysql Workbench, i created another crawler but this time configured the crawler to crawl the RDS. By doing so it identifies those new tables and gets me tables with data versus tables without data in the same enironment with the empty tables.
I needed one more job, which would be an ETL job to copy data from `formula-1-database` to the empty tables in Mysql Workbench. To achieve this i used AWS Glue Studio.

![alt text](<10 ETL Jobs.png>)

### Phase 5 - Getting Data from Database for Data Analysis
Finally what was left was simply running some queires from the database to explore the data, create some views that gave results for desired metrics before fetching the data from a powerful visualization tool. I've shared the whole process overview in the architecture diagram below and also a sample visualization i did using Power BI.

Thanks!

![alt text](<11 Project Architecture.png>)

<video controls src="14 F1_Media.mp4" title="Title"></video>


